# Extraction des articles du serveur Erudit et stockage en mémoire locale

## **OBJECTIF GENERAL**

Etablir un code propre et optimisé pour extraire le contenu et métadonnées des articles XML du serveur Erudit

## **OBJECTIFS SPECIFIQUES**

- Indexer les journaux d'intérêt dont on souhaite récupérer les articles
- Former un corpus associant le texte et les métadonnées pour chaque article

## **METHODE**

**I. Indexation des revues, journaux et articles d'intérêt**

1 - Création de listes contenant les indexes des journaux, revues et articles d'intérêt

2 - Création de dossiers de stockage en local et stockage des objets requests (.xml)

**II. Formation du Corpus**

1) Récupération du contenu XML des articles sous forme d'objet soup

2) Création d'un tuple pour chaque article à partir de l'objet soup: (texte, {métadonnées : revue, auteur, date})

3) Aggrégation des tuples sous la forme d'une liste représentant l'ensemble d'une revue

# **PIPELINE**

**INPUT** : 

Fichiers XML du serveur Erudit

**OUTPUT** : 
- Listes d'indexation : ref_journal - ref_revue - ref_article

- Contenu articles et métadonnées : Liste de tuples de la forme (texte de l'article, {ditionnaire de métadonnées})

### Import des bibliothèques

In [ ]:
import os
import pathlib
import sys
import random

import urllib.request
import requests
from bs4 import BeautifulSoup

import pickle
import spacy
import textacy
from tqdm import tqdm_notebook

-------------------

# I. INDEXATION (à faire une seule fois)

-------------------

**OUTPUT DE CETTE PARTIE**

In [ ]:
# Listes pour stocker les chemins des différents articles
% store -r ref_journal
% store -r ref_revue
% store -r ref_article

Sélection des revues d'intérêt

Critère de sélection: 
- **Thématiques**: Economie, innovation, risque, management, finance, commerce international (lien avec intérêts de recherche de Catherine)
- **Langue** : Français
- **Periode de publication** : au moins 40 ans (pour les modèles dynamiques)

Revues =  Actualité Economique, Relations Industrielles et Etudes Internationales

NB : La liste ref_journal peut être modifiée en fonction des revues d'intérêt. Il suffit d'ajouter le chemin correspondant vers le serveur Erudit.

**REF_JOURNAL**

In [ ]:
# Stockage du chemin de chaque revue vers le serveur Erudit
ref_journal = ['/erudit/ae49', '/erudit/ri17', '/erudit/ei50']
% store ref_journal

**REF_REVUE**

In [ ]:
# Stockage des id des revues pour chaque journal
ref_revue = [] # liste de liste où chaque élément est du type [index_revue,index_numero] = numero de publication
for index_journal in range(len(ref_journal)):
    h_ref = []
    path = 'https://fichiers.erudit.org/cyber/'+ ref_journal[index_journal]
    r = requests.get(path, auth=('mgagnon', 'iathe6eiyaiy6Eic'))
    soup = BeautifulSoup(r.text, "lxml")
    all_links = soup.find_all("a")
    for link in all_links:
        h_ref.append(link.get("href"))
    ref_revue.append(h_ref)

# on enlève les balises inutiles au début des documents HTML
for j in range (5):
    for i in range(len(ref_journal)):
        ref_revue[i].pop(0)
% store ref_revue

**REF_ARTICLE**

In [ ]:
# stockage des id des articles pour chaque revue
total_folders = []
for i in range(len(ref_revue)):
    total_folders.append(len(ref_revue[i]))

ref_article = [] #liste de 3 colonnes avec [Revue][Numero de Publication][Article] = numéro article
for index_journal in tqdm_notebook(range(len(ref_journal))):
    h_ref_bis = []
    for j in tqdm_notebook(range(total_folders[index_journal])):
        h_ref = []
        path = 'https://fichiers.erudit.org/cyber'+ ref_journal[index_journal] + '/' + ref_revue[index_journal][j]
        r = requests.get(path, auth=('mgagnon', 'iathe6eiyaiy6Eic'))
        soup = BeautifulSoup(r.text, "lxml")
        new_links = soup.find_all("a")
        for link in new_links:
            h_ref.append(link.get("href"))  
        h_ref_bis.append(h_ref)
    ref_article.append(h_ref_bis)

for k in range(5):
    for index_journal in range(len(ref_journal)):
        for j in range(total_folders[index_journal]):
            ref_article[index_journal][j].pop(0)
% store ref_article

-------------------

# II.FORMATION DU CORPUS

-------------------

**OUTPUT DE CETTE PARTIE**

In [ ]:
# Récupération d'une liste de dictionnaire du type {'texte' : texte article brut , 'metadata': {'title' : titre, 'typeart': , 'lang': , 'annee': ,'info_auteurs': [(prenom, nom, affiliation)]}}
%store -r Corpus_AE
%store -r Corpus_EI
%store -r Corpus_RI

### 1. Récupération XML en objets soup

In [ ]:
index_journaux = {'AE' : 0, 'RI' : 1, 'EI' : 2}

**USAGE:**

- Premier usage : créer le dossier "Variables résultats"

- *request_xml* doit être lancée une unique fois pour chaque revue (écriture des xml en local)

- *get_soup_from_xml* doit être lancée à chaque utlisation du notebook et pour chaque revue (récupération des objets soup)
- De même, les listes liste_AE, liste_EI, liste_RI doivent être recrées à chaque utilisation du notebook

In [ ]:
def request_xml(ref_revue, ref_article, ref_journal, journal ='AE'):
    """ Fonction pour stocker les requests pour tous les articles d'intérêt depuis le serveur Erudit"""
    index_journal = index_journaux[journal]
    for index_revue in tqdm_notebook(range(len(ref_revue[index_journal]))):
        for index_article in range(len(ref_article[index_journal][index_revue])):
            # Récupération du fichier XML sur le serveur et création d'un objet BeautifulSoup
            article_path = 'https://fichiers.erudit.org/cyber' + ref_journal[index_journal] + '/' + ref_revue[index_journal][index_revue] + ref_article[index_journal][index_revue][index_article].replace("/", "") 
            r = requests.get(article_path + '/ERUDITXSD300.xml', auth=('mgagnon', 'iathe6eiyaiy6Eic')) 
            # Ecriture en local des objets request
            with open('Variables résultats/requests_' + journal + '/' + ref_revue[index_journal][index_revue].replace("/","") 
                      + '_' + ref_article[index_journal][index_revue][index_article].replace("/", "")  +'.xml', 'w', encoding ='utf-8') as file :
                file.write(r.text)
                file.close()
    print("Stockage des fichiers requests terminé")

In [ ]:
# Stockage des fichiers requests en local : à faire tourner une seule fois (=1h20 min)
request_xml(ref_revue, ref_article, ref_journal, journal= 'AE')
request_xml(ref_revue, ref_article, ref_journal, journal= 'RI')
request_xml(ref_revue, ref_article, ref_journal, journal= 'EI')

#### **USAGE** : FAIRE TOURNER LES 4 CELLULES CI-DESSOUS A CHAQUE OUVERTURE DU NOTEBOOK

In [ ]:
# Listes pour stocker les chemins des différents articles
% store -r ref_journal
% store -r ref_revue
% store -r ref_article

In [ ]:
index_journaux = {'AE' : 0, 'RI' : 1, 'EI' : 2}

In [ ]:
def get_soup_from_xml (ref_revue, ref_article, ref_journal, journal = 'AE'):
    soup_articles = []
    index_journal = index_journaux[journal]
    for index_revue in tqdm_notebook(range(len(ref_revue[index_journal]))):
        for index_article in range(len(ref_article[index_journal][index_revue])):
            article_path = 'https://fichiers.erudit.org/cyber' + ref_journal[index_journal] + '/' + ref_revue[index_journal][index_revue] + ref_article[index_journal][index_revue][index_article].replace("/", "") 
            with open("Variables résultats/requests_" + journal +'/' + ref_revue[index_journal][index_revue].replace("/","") 
                      + '_' + ref_article[index_journal][index_revue][index_article].replace("/", "")  +'.xml', 'r', encoding ='utf-8') as r :
                soup_articles.append((article_path, BeautifulSoup(r, "lxml")))
    return soup_articles

In [ ]:
# Récupérer les objets soup : à refaire à chaque fois (=5 min)
soup_articles_AE = get_soup_from_xml(ref_revue, ref_article, ref_journal, journal= 'AE')
soup_articles_RI = get_soup_from_xml(ref_revue, ref_article, ref_journal, journal= 'RI')
soup_articles_EI = get_soup_from_xml(ref_revue, ref_article, ref_journal, journal= 'EI')

### 2. Formation du Corpus

**OUTPUT** : liste de tuples dont les éléments sont du type (texte, {métadonnées})

In [ ]:
def create_corpus (soup_articles):
    """ Récupérer le texte brut et les métadonnées d'intérêt pour chaque article"""
    Corpus = []
    for index in tqdm_notebook(range(len(soup_articles))):
        soup = soup_articles[index][1] # récupération de l'objet soup
        try:
            if soup == None: pass
            
            # Récupération du prénom, nom et affiliation des auteurs de l'article
            auteurs = []
            for auteur in soup.find_all('auteur'):
                try: #test si l'information est complète, sinon on passe
                    auteurs.append((auteur.prenom.text,auteur.nomfamille.text,auteur.affiliation.text))
                except:
                    auteurs = None
                    break
            # métadonnées du XML que l'on garde en mémoire    
            metadata = {}
            metadata['title'] =  soup.titre.text if soup.titre != None else None
            metadata['typeart'] =  soup.article['typeart'] if soup.article != None else None
            metadata['lang'] =  soup.article['lang'] if soup.article != None else None
            metadata['traitement'] =  soup.article['qualtraitement'] if soup.article != None else None
            metadata['annee'] =  soup.annee.text if soup.annee != None else None
            metadata['info_auteurs'] =  auteurs 

            # Ajout du texte et des métadonnées au sein d'un tuple représentant l'article
            Corpus.append({'texte' : soup.corps.text, 'metadata': metadata, 'URL' : soup_articles[index][0]})
            
        except AttributeError as error:
            print(soup_articles[index][0])
    # On retourne la liste de tupes pour la revue concernée
    return Corpus

In [ ]:
# Running time < 3min
Corpus_AE = create_corpus(soup_articles_AE)
Corpus_EI = create_corpus(soup_articles_EI)
Corpus_RI = create_corpus(soup_articles_RI)

In [ ]:
%store Corpus_AE
%store Corpus_EI
%store Corpus_RI